# Points à vérifier dans mon code: 

In [1]:
from DL_class import InvalidDatesCleaner,DatesVerifFeatureVect
# Trouver un moyen d'uniformiser 'time_step_per_hour' pour toute les données, genre dénominateur commun etc.
dates_verif_obj = DatesVerifFeatureVect(dataset.df_dates, Weeks = args.W, Days = args.D, historical_len = args.H, step_ahead = args.step_ahead, time_step_per_hour = dataset.time_step_per_hour)
dataset.get_feature_vect(invalid_dates)
# ========== .... ==========

# Passage dans split Tensor: 
# >>>>  Consiste à Normalizer un input (Ici 'U') selon une certaine référence (ici train_input)
# >>>>  Puis de renovyer les U_train/U_valid/U_test 
# >>>>
# >>>>   Pour se faire, ils calculent d'abord les stats (min,max,mean,std) sur le tensor de référence train_input
# >>>>   Puis, à chacun des split U_Train/U_Valid/U_Test, normalisation selon les statistiques.


# ==== ATTENTION il faudra sans doute récupérer les stats simplement et pas normalizer la ref
# ==== Calcul inutile ici 


# ========= PERSONAL_INPUT =========
# >>>> 
# >>>> PB_1 : Il faut absolument pouvoir prendre en compte les invalid dates de tout le monde.
# >>>> 1. Il le faire en amont. i.e lorsqu'on load les données, on load aussi les invalid dates associées
# >>>> 2. Puis on fait l'union des invalid dates
# >>>> 3. Après ça on train/valid/test split normalize tout.
# >>>> 
# >>>> PB_2: Idem pour le coverage pour que ça corresponde à tout le monde.  
# >>>> 
# >>>> PB_3 : Génère des input de taille différentes selon l'historique demandé. 
# >>>> Cela ne va pas convenir si jamais je travail avec du calendar class ou autre.
# >>>> 1. On peut donc enregistrer l'ensemble des invalid-dates
# >>>> 2. Créer les séquences/input_tensor associées 
# >>>> 3. Pour chaque donnée, on ne prend peut être pas en compte les même historiques. 
# >>>>    Donc on peut faire une liste de date 'D' correspondant à l'intersection de toute les target (df_verif[-1] ...?)
# >>>> 4. Puis on extrait l'ensemble des sequences de chaque Data sur D 


NameError: name 'DatesVerifFeatureVect' is not defined

### Resultats de Normalization qu'on est censé conserver : 
**Tackling Subway Data:**
- U_train min:  tensor(0.) U_train max:  tensor(1.)
- U_valid min:  tensor(0.0447) U_valid max:  tensor(4.1455)
- U_test min:  tensor(0.) U_test max:  tensor(4.1892)

**Tackling NetMob Data:** 
- U_train min:  tensor(0.) U_train max:  tensor(1.)
- U_valid min:  tensor(-0.1006) U_valid max:  tensor(1.0808)
- U_test min:  tensor(-0.1259) U_test max:  tensor(1.1083)

# Entrainement avec Subway pour voir si tout est bien conservé 

In [1]:
from dataset import TrainValidTest_Split_Normalize,PersonnalInput
from loader import DictDataLoader
from trainer import Trainer 

from plotting_bokeh import generate_bokeh
from calendar_class import get_time_slots_labels
import os
import pandas as pd 
import torch
import pickle 
import numpy as np 

from paths import folder_path,file_name
from config import get_args
from utilities_DL import get_DataSet_and_invalid_dates, match_period_coverage_with_netmob,get_loss,load_model_and_optimizer,get_model_loss_args_emb_opts


def load_subway_in(folder_path,file_name,args,coverage):
    '''Tackling Subway_in data'''
    dataset,invalid_dates = get_DataSet_and_invalid_dates(args.abs_path, folder_path,file_name,
                                                        args.W,args.D,args.H,args.step_ahead,
                                                        single_station = False,coverage_period = coverage)

    subway_ds = PersonnalInput(invalid_dates,args, tensor = dataset.raw_values, dates = dataset.df_dates,
                            time_step_per_hour = dataset.time_step_per_hour,Weeks = args.W, Days = args.D, historical_len = args.H,step_ahead = args.step_ahead,minmaxnorm = True ,dims=[0])
    subway_ds.preprocess(args.train_prop,args.valid_prop,args.test_prop)
    return(subway_ds,dataset,invalid_dates)

def load_calendar(subway_ds):
    '''Tackling Calendar Data''' 
    time_slots_labels,dic_class2rpz,dic_rpz2class,nb_words_embedding = get_time_slots_labels(subway_ds)
    tensor_limits_keeper = subway_ds.tensor_limits_keeper

    dict_calendar_U_train,dict_calendar_U_valid,dict_calendar_U_test = {},{},{}
    for calendar_class in [0,1,2,3]:
        calendar_tensor = time_slots_labels[calendar_class] #args.calendar_class

        splitter = TrainValidTest_Split_Normalize(calendar_tensor,
                                    first_train = tensor_limits_keeper.first_train_U, last_train= tensor_limits_keeper.last_train_U,
                                    first_valid= tensor_limits_keeper.first_valid_U, last_valid = tensor_limits_keeper.last_valid_U,
                                    first_test = tensor_limits_keeper.first_test_U, last_test = tensor_limits_keeper.last_test_U)

        train_tensor_ds,valid_tensor_ds,test_tensor_ds = splitter.split_normalize_tensor_datasets(normalizer = None)
        calendar_U_train,calendar_U_valid,calendar_U_test = train_tensor_ds.tensor,valid_tensor_ds.tensor,test_tensor_ds.tensor
        dict_calendar_U_train[calendar_class] = calendar_U_train
        dict_calendar_U_valid[calendar_class] = calendar_U_valid
        dict_calendar_U_test[calendar_class] = calendar_U_test
    return(dict_calendar_U_train,dict_calendar_U_valid,dict_calendar_U_test,dic_class2rpz,dic_rpz2class,nb_words_embedding)


def load_netmob_data(dataset,invalid_dates,args,save_folder,ref_subway = None):
    '''Load NetMob Data:
    outputs:
    --------
    # NetMob Tensor : [T,N,C,H,W]
    # dims : [0,3,4] #[0,-2,-1]  -> dimension for which we want to retrieve stats 
    '''
    try :
        netmob_T = torch.stack([torch.load(f"{save_folder}station_{station}.pt") for station in ref_subway.COD_TRG])
        netmob_T = netmob_T.permute(1,0,*range(2, netmob_T.dim()))

    except:
        netmob_T = torch.randn(dataset.length,40,2,22,22)
        print("Load des données NetMob .pt impossible. Création d'un random Tensor")

    print('Init NetMob Dataset: ', netmob_T.size())
    print('Number of Nan Value: ',torch.isnan(netmob_T).sum())
    print('Total Number of Elements: ', netmob_T.numel(),'\n')

    NetMob_ds = PersonnalInput(invalid_dates,args, tensor = netmob_T, dates = dataset.df_dates,
                           time_step_per_hour = dataset.time_step_per_hour,Weeks = args.W, Days = args.D, historical_len = args.H,step_ahead = args.step_ahead,minmaxnorm = True,dims =[0,3,4])
    NetMob_ds.preprocess(args.train_prop,args.valid_prop,args.test_prop)

    return(NetMob_ds)


def add_contextual_data(dataset_names,subway_ds,NetMob_ds,dict_calendar_U_train,dict_calendar_U_valid,dict_calendar_U_test):
    # === Define DataLoader : 
    contextual_tensors,positions = {},{}
    if 'calendar' in dataset_names:
        contextual_tensors = {f'calendar_{calendar_class}': {'train': dict_calendar_U_train[calendar_class],
                                        'valid': dict_calendar_U_valid[calendar_class],
                                        'test': dict_calendar_U_test[calendar_class]} for calendar_class in dict_calendar_U_train.keys()
                                        } 
        pos_calendar = list(contextual_tensors.keys()).index(f'calendar_{args.calendar_class}')
        pos_calibration_calendar = list(contextual_tensors.keys()).index(f'calendar_{0}')
        positions['calendar'] = pos_calendar
        positions['calibration_calendar'] = pos_calibration_calendar

    if 'netmob' in dataset_names:
        contextual_tensors.update({'netmob': {'train': NetMob_ds.U_train,
                                        'valid': NetMob_ds.U_valid,
                                        'test': NetMob_ds.U_test}
                                        }
                                        )
        
        pos_netmob = list(contextual_tensors.keys()).index('netmob')
        positions['netmob'] = pos_netmob



    subway_ds.contextual_tensors = contextual_tensors
    subway_ds.get_dataloader()

    return(subway_ds,positions)

def load_everything(dataset_names,folder_path,file_name,args,coverage,data_folder_path):
    subway_ds,dataset,invalid_dates = load_subway_in(folder_path,file_name,args,coverage)
    # Calendar: 
    if 'calendar' in dataset_names:
        dict_calendar_U_train,dict_calendar_U_valid,dict_calendar_U_test,dic_class2rpz,dic_rpz2class,nb_words_embedding = load_calendar(subway_ds)
        args.time_embedding = True
    else:
        dict_calendar_U_train,dict_calendar_U_valid,dict_calendar_U_test,dic_class2rpz,dic_rpz2class,nb_words_embedding = None,None,None,None,None,None
        args.time_embedding = False
    # ...

    # Netmob: 
    if 'netmob' in dataset_names:
        NetMob_ds = load_netmob_data(dataset,invalid_dates,args,data_folder_path,ref_subway = None)
    else:
        NetMob_ds = None
    # ...
    
    subway_ds,positions = add_contextual_data(dataset_names,subway_ds,NetMob_ds,dict_calendar_U_train,dict_calendar_U_valid,dict_calendar_U_test)
    return(args,subway_ds,positions,nb_words_embedding,dic_class2rpz)

def get_small_ds(small_ds,coverage,args):
    if small_ds:
        coverage = coverage[:100]
        args.W = 0
        args.D = 0
        print('Seulement les 100 premiers time-slots sont utilisés.')
    return(coverage,args)

'pynvml' is not available on this environment.


In [2]:
# Load config
model_name = 'STGCN' #'CNN'
netmob = True
args = get_args(model_name)
#args = get_args(model_name = model_name,learn_graph_structure = True)  # MTGNN

# Modification : 
args.K_fold = 1
args.ray = False
args.epochs = 15
args.loss_function_type = 'MSE'

small_ds = False

coverage = match_period_coverage_with_netmob(file_name)

if torch.cuda.is_available():
    data_folder_path = '../../../data/' 
else:
    data_folder_path = '../../Data/'



(coverage,args) = get_small_ds(small_ds,coverage,args)
dataset_names = ['subway_in','calendar']#['subway_in','netmob','calendar']

args,subway_ds, positions,nb_words_embedding,dic_class2rpz = load_everything(dataset_names,folder_path,file_name,args,coverage,data_folder_path)
# Load Model, Optimizer, Scheduler: 
loss_function,model,optimizer,scheduler,args_embedding = get_model_loss_args_emb_opts(args,nb_words_embedding,dic_class2rpz,n_vertex = subway_ds.raw_values.size(1))
# Load trainer: 
trainer = Trainer(subway_ds,model,args,optimizer,loss_function,scheduler = None,args_embedding  =args_embedding,dic_class2rpz = dic_class2rpz,show_figure = True,positions = positions)# Ajoute dans trainer, if calibration_prop is not None .... et on modifie le dataloader en ajoutant un clabration set
# Train Model 
trainer.train_and_valid(mod = 1000)  # Récupère les conformity scores sur I1, avec les estimations faites precedemment 

Time-step per hour: 4.0
coverage period: 2019-03-16 00:00:00 - 2019-05-31 23:45:00

 Tackling Training Set
Values with issues:  0.000%
Regular Values that we have to set to 0:  0.000%

 Tackling Validation Set
Values with issues:  0.000%
Regular Values that we have to set to 0:  0.000%

 Tackling Testing Set
Values with issues:  0.000%
Regular Values that we have to set to 0:  0.000%

 Tackling Training Set
Values with issues:  0.000%
Regular Values that we have to set to 0:  0.000%

 Tackling Validation Set
Values with issues:  0.000%
Regular Values that we have to set to 0:  0.000%

 Tackling Testing Set
Values with issues:  0.000%
Regular Values that we have to set to 0:  0.000%

U size:  torch.Size([5662, 40, 8]) Utarget size:  torch.Size([5662, 40, 1])
U_train size:  torch.Size([2934, 40, 8]) Utarget_train size:  torch.Size([2934, 40, 1])
U_valid size:  torch.Size([978, 40, 8]) Utarget_valid size:  torch.Size([978, 40, 1])
U_test size:  torch.Size([979, 40, 8]) Utarget_test size: 

/Users/romainrochas/opt/anaconda3/envs/preprocessingclone/lib/python3.7/site-packages/torch/cuda/amp/grad_scaler.py:118: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")
/Users/romainrochas/opt/anaconda3/envs/preprocessingclone/lib/python3.7/site-packages/torch/amp/autocast_mode.py:202: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


epoch: 0 
 min\epoch : 0.18
Estimated time for training: 2.8min 
Training Throughput:307.30 sequences per seconds
>>> Training complete in: 0:02:47.575413
>>> Training performance time: min 0.16910219192504883 avg 0.2022719383239746 seconds (+/- 0.020194933588228735)
>>> Loading performance time: min 0.0006051063537597656 avg 0.11128964437835517 seconds (+/- 0.17874503792503263)
>>> Forward performance time: 0.05674124525316568 seconds (+/- 0.009619733500805064)
>>> Backward performance time: 0.15076632977224053 seconds (+/- 0.015603531437756163)
>>> Plotting performance time: 1.1989048549107143e-05 seconds (+/- 3.6087652850581235e-05)
>>> Saving performance time: 0.0047625541687011715 seconds (+/- 0.0023080803558375245)
>>> PI-tracking performance time: 3.491129193987165e-06 seconds (+/- 4.819792235948326e-07)
>>> Scheduler-update performance time: 2.418245588030134e-06 seconds (+/- 5.243439101607682e-07)
Proportion of time consumed for Loading: 35.1%
Proportion of time consumed for F

In [3]:
if args.loss_function_type == 'quantile':
    Q = trainer.conformal_calibration(args.alpha,conformity_scores_type =args.conformity_scores_type, quantile_method = args.quantile_method)  # calibration for PI 90%
else:
    Q = None
station = 0
pi,pi_cqr = generate_bokeh(trainer,trainer.dataloader,
                                    trainer.dataset,Q,args,trainer.dic_class2rpz,
                                    station = station,
                                    show_figure = True,
                                    save_plot = False
                                    )

Loading BokehJS ...

# TEST avec plusieurs contextual Tensors: 

In [ ]:
contextual_tensors = {'calendar': {'train': calendar_U_train,
                                   'valid': calendar_U_valid,
                                   'test': calendar_U_test},
                      
                                 }




## Essais avec NetMob

In [4]:
T = netmob_T.size(0)

# Tackle a specific fold : 
netmob_T1 = netmob_T[:100]

# Init :
dims = [0,3,4]#[0,-2,-1]
minmaxnorm = True
standardize = False

# ============ Load Train/Valid/Test Indices and removed forbidden dates : ============
# invalid dates = 
# invalid_indices = get_indices_from_dates(invalid)
indices = np.arange(T)
np.random.shuffle(indices)
invalid_indices = indices[:100]

# Get Split indices :
train_indices = np.arange(50)
valid_indices = np.arange(60,70)
test_indices = np.arange(80,100)

# Remove invalid_dates from indices :
cleaner = InvalidDatesCleaner(invalid_indices = invalid_indices)

# >>>> Le cleaner prend un ensemble d'indice qui est interdit (les indices correspondant au dates qui sont interdites)
# >>>> On lui passe ensemble un ensemble d'indices (ex : ceux qui correspondent à l'extraction de U_train)
# >>>> Pour lequels il va masqué/supprimer les indices interdit
# >>>> On ce sert ensuite de cet ensemble pour extraire le bon U_train, avec les valeurs qui sont effectivement interdite. 


train_indices = cleaner.clean_indices(train_indices)
valid_indices = cleaner.clean_indices(valid_indices)
test_indices = cleaner.clean_indices(test_indices)
# ============ .......................................................... ============

# Load Splitter Object
splitter = TrainValidTest_Split_Normalize(netmob_T1,dims,train_indices, valid_indices, test_indices,minmaxnorm=minmaxnorm,standardize=standardize)

# Split DataSet and Normalize accoding Stats from Training Set 
train_dataset,valid_dataset,test_dataset = splitter.load_normalize_tensor_datasets()
# Define DictDataLoader :


 Tackling Training Set


IndexError: min(): Expected reduction dim 2 to have non-zero size.